In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [48]:
# Load the CSV data
data = pd.read_csv("data_moods.csv")
truck_data = pd.read_csv("data.csv")

In [50]:


numeric_cols = ['danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'valence',  'speechiness']

# Round to two decimal places and multiply by 100
for col in numeric_cols:
    data[col] = (data[col] * 100).round(0)
    truck_data[col] = (truck_data[col] * 100).round(0)

In [4]:
data.head()

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,1999,1999,Prince,2H7PHVdQ3mXqEHXcvclTB0,1982-10-27,68,379266,87.0,14.0,73.0,0.0,8.0,62.0,-8.20,8.0,118.52,5,4,Happy
1,23,23,Blonde Redhead,4HIwL9ii9CcXpTOTzMq0MP,2007-04-16,43,318800,38.0,2.0,83.0,20.0,15.0,17.0,-5.07,5.0,120.26,8,4,Sad
2,9 Crimes,9,Damien Rice,5GZEeowhvSieFDiR8fQ2im,2006-11-06,60,217946,35.0,91.0,14.0,0.0,9.0,12.0,-15.33,3.0,136.17,0,4,Sad
3,99 Luftballons,99 Luftballons,Nena,6HA97v4wEGQ5TUClRM0XLc,1984-08-21,2,233000,47.0,9.0,44.0,0.0,11.0,59.0,-12.86,6.0,193.10,4,4,Happy
4,A Boy Brushed Red Living In Black And White,They're Only Chasing Safety,Underoath,47IWLfIKOKhFnz1FUEUIkE,2004-01-01,60,268000,42.0,0.0,93.0,0.0,14.0,44.0,-3.60,11.0,169.88,1,4,Energetic


In [52]:
features = ['danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness', 'tempo', 'key']
X = data[features]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['mood'])

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

X_pre= truck_data[features]

In [6]:
X_train

,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key
278,46.0,92.0,18.0,0.0,12.0,18.0,-14.85,4.0,80.75,0
110,69.0,10.0,96.0,1.0,8.0,45.0,-3.66,11.0,123.98,1
82,74.0,95.0,31.0,0.0,19.0,36.0,-7.83,7.0,111.20,4
51,57.0,84.0,16.0,0.0,9.0,9.0,-14.40,3.0,109.92,6
218,28.0,85.0,26.0,0.0,10.0,11.0,-9.87,3.0,98.66,11
...,...,...,...,...,...,...,...,...,...,...
71,60.0,2.0,90.0,68.0,10.0,14.0,-5.98,5.0,127.97,5
106,70.0,0.0,68.0,0.0,6.0,55.0,-4.18,6.0,150.12,2
270,27.0,99.0,0.0,89.0,9.0,15.0,-33.73,5.0,132.58,7
435,30.0,17.0,54.0,80.0,13.0,19.0,-10.64,5.0,142.72,7


In [53]:
# Convert features to a textual format
text_data_train = X_train.apply(lambda x: ', '.join(f"{feature}: {x[feature]}" for feature in features), axis=1).tolist()
text_data_val = X_val.apply(lambda x: ', '.join(f"{feature}: {x[feature]}" for feature in features), axis=1).tolist()
text_data_pre=X_pre.apply(lambda x: ', '.join(f"{feature}: {x[feature]}" for feature in features), axis=1).tolist()


In [54]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Now, you can tokenize your data
train_encodings = tokenizer(text_data_train, truncation=True, padding=True, return_tensors="pt", max_length=128)
val_encodings = tokenizer(text_data_val, truncation=True, padding=True, return_tensors="pt", max_length=128)
pred_encoding=tokenizer(text_data_pre, truncation=True, padding=True, return_tensors="pt", max_length=128)

In [11]:
from transformers import GPT2ForSequenceClassification, Trainer, TrainingArguments

#model = GPT2ForSequenceClassification.from_pretrained("distilgpt2", num_labels=4)
#model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=4)
model = GPT2ForSequenceClassification.from_pretrained("gpt2-large", num_labels=4)
#model = GPT2ForSequenceClassification.from_pretrained("gpt2-xl", num_labels=4)
model.config.pad_token_id = tokenizer.eos_token_id


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from torch.utils.data import Dataset
import torch
from torch.utils.data import Dataset
import numpy as np

class MoodDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure labels are of type torch.long
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = MoodDataset(train_encodings, y_train)
val_dataset = MoodDataset(val_encodings, y_val)


In [13]:
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    pred = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, pred)}

In [14]:
training_args = TrainingArguments(
    output_dir='./results',   # Add this line to specify the output directory
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)



In [15]:
trainer.train()
trainer.evaluate()

  0%|          | 0/690 [00:00<?, ?it/s]C:\Users\Ze\AppData\Local\Temp\ipykernel_23024\3041598397.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  0%|          | 1/690 [00:01<13:03,  1.14s/it]C:\Users\Ze\AppData\Local\Temp\ipykernel_23024\3041598397.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  2%|▏         | 11/690 [00:03<02:39,  4.25it/s]

{'loss': 1.8957, 'learning_rate': 4.9275362318840584e-05, 'epoch': 0.14}


  3%|▎         | 21/690 [00:05<02:38,  4.23it/s]

{'loss': 1.3993, 'learning_rate': 4.855072463768116e-05, 'epoch': 0.29}


  4%|▍         | 31/690 [00:08<02:38,  4.15it/s]

...

{'eval_loss': 1.7938377857208252, 'eval_accuracy': 0.7753623188405797, 'eval_runtime': 0.6624, 'eval_samples_per_second': 208.319, 'eval_steps_per_second': 27.172, 'epoch': 10.0}


100%|██████████| 690/690 [04:06<00:00,  2.80it/s]
C:\Users\Ze\AppData\Local\Temp\ipykernel_23024\3041598397.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'train_runtime': 246.8062, 'train_samples_per_second': 22.204, 'train_steps_per_second': 2.796, 'train_loss': 0.40140211117045577, 'epoch': 10.0}


100%|██████████| 18/18 [00:00<00:00, 27.65it/s]


{'eval_loss': 1.5577775239944458,
 'eval_accuracy': 0.782608695652174,
 'eval_runtime': 0.693,
 'eval_samples_per_second': 199.14,
 'eval_steps_per_second': 25.975,
 'epoch': 10.0}

In [55]:
from torch.nn.functional import softmax

# Ensure your model is on the correct device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set your batch size
batch_size = 5  # Adjust this to a smaller number if your GPU memory is limited

# Initialize an empty list to store logits from each batch
all_logits = []

# Process the data in batches
for i in range(0, len(pred_encoding['input_ids']), batch_size):
    # Extract the batch from pred_encoding
    batch = {key: val[i:i + batch_size] for key, val in pred_encoding.items()}
    
    # Move batch to the same device as the model
    batch = {k: v.to(device) for k, v in batch.items()}
    
    # Forward pass without gradient calculation (to save memory)
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        all_logits.append(logits)

# Now, all_logits contains logits from each batch. You can concatenate them and process further.
all_logits = torch.cat(all_logits, dim=0)

# Apply softmax to convert logits to probabilities (if needed)
probabilities = softmax(all_logits, dim=1)


In [56]:
# Convert probabilities tensor to CPU and then to a NumPy array
probabilities_cpu = probabilities.cpu().numpy()

# Now you can use the label_encoder to inverse transform the predictions
mood_predictions = label_encoder.inverse_transform(probabilities_cpu.argmax(axis=1))

# mood_predictions now contains the original mood labels


In [57]:
truck_data['predicted_mood'] = mood_predictions

In [60]:
truck_data['predicted_mood']

0          Calm
1           Sad
2          Calm
3           Sad
4           Sad
          ...  
170648    Happy
170649    Happy
170650    Happy
170651    Happy
170652    Happy
Name: predicted_mood, Length: 170653, dtype: object

In [61]:
truck_data.to_csv("data_music_with_predictions.csv", index=False)